In [ ]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import tensorflow as tf
import tensorflow_addons as tfa
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from pytorch_tabnet.tab_model import TabNetRegressor
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import functional as F

In [ ]:
x_train = pd.read_csv('../input/lish-moa/train_features.csv')
x_test = pd.read_csv('../input/lish-moa/test_features.csv')
y_train = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
y_train_non = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
y_train_non.shape

In [ ]:
trt_cp_index=x_train[x_train['cp_type']=='trt_cp'].index

In [ ]:
x_test_checkpoint=x_test.copy()

In [ ]:
x_test_checkpoint.head()

In [ ]:
x_train.head()

In [ ]:
x_train=x_train.drop(['sig_id'],1)
x_test=x_test.drop(['sig_id'],1)
y_train=y_train.drop(['sig_id'],1)
x_train.head()

In [ ]:
x_train=x_train.drop(['cp_type'],1)
x_test=x_test.drop(['cp_type'],1)

In [ ]:
x_train['cp_time'] = x_train['cp_time'].map({24:0,48:1,72:2})
x_train.cp_time.unique()
x_test['cp_time'] = x_test['cp_time'].map({24:0,48:1,72:2})
x_test.cp_time.unique()

In [ ]:
x_train['cp_dose'] = x_train['cp_dose'].map({"D1":0,"D2":1})
x_train.cp_dose.unique()
x_test['cp_dose'] = x_test['cp_dose'].map({"D1":0,"D2":1})
x_test.cp_dose.unique()

In [ ]:
x_train

In [ ]:
genes=[col for col in x_train.columns if col.startswith("g-")]
cells=[col for col in x_train.columns if col.startswith("c-")]

In [ ]:
a_train=x_train.copy()
a_test=x_test.copy()

In [ ]:
x_train=a_train.copy()
x_test=a_test.copy()

In [ ]:
scaler = QuantileTransformer(n_quantiles=100,random_state=42, output_distribution="normal")
scaled_data=scaler.fit_transform(x_train[genes+cells])
x_train=pd.concat([x_train.iloc[:,:2],pd.DataFrame(scaled_data,columns=genes+cells)],1)
# scaled_data=scaler.transform(x_test[genes+cells])
# x_test=pd.concat([x_test.iloc[:,:2],pd.DataFrame(scaled_data,columns=genes+cells)],1)


transformer = QuantileTransformer(n_quantiles=100,random_state=42, output_distribution="normal")
scaled_data=scaler.fit_transform(x_test[genes+cells])
x_test=pd.concat([x_test.iloc[:,:2],pd.DataFrame(scaled_data,columns=genes+cells)],1)

In [ ]:
x_test.iloc[:,2:].sum().sum()

In [ ]:
# genes = [cols for cols in x_train.columns if cols.startswith('c-')]
# data=pd.concat([pd.DataFrame(x_train[genes]),pd.DataFrame(x_test[genes])])
# g_data=PCA(0.95,random_state=42).fit_transform(data)

In [ ]:
# g_data.sum()

In [ ]:
data=pd.concat((x_train[genes],x_test[genes]))
g_data=PCA(n_components=0.95,random_state=44).fit_transform(data)

g_train=g_data[:x_train.shape[0]]
g_test=g_data[x_train.shape[0]:]

g_train_df=pd.DataFrame(g_train,columns=[f'pca_g-{i}' for i in range(g_data.shape[1])])
g_test_df=pd.DataFrame(g_test,columns=[f'pca_g-{i}' for i in range(g_data.shape[1])])


data=pd.concat((x_train[cells],x_test[cells]))
c_data=PCA(n_components=0.95,random_state=44).fit_transform(data)

c_train=c_data[:x_train.shape[0]]
c_test=c_data[x_train.shape[0]:]

c_train_df=pd.DataFrame(c_train,columns=[f'pca_c-{i}' for i in range(c_data.shape[1])])
c_test_df=pd.DataFrame(c_test,columns=[f'pca_c-{i}' for i in range(c_data.shape[1])])

In [ ]:
X_train=pd.concat((x_train,g_train_df,c_train_df),axis=1)
X_test=pd.concat((x_test,g_test_df,c_test_df),axis=1)

In [ ]:
from sklearn.feature_selection import VarianceThreshold
varthre = VarianceThreshold(0.8)
train_varthre=varthre.fit_transform(X_train.iloc[:,2:])
test_varthre=varthre.transform(X_test.iloc[:,2:])

In [ ]:
X_train=pd.concat([X_train.iloc[:,:2],pd.DataFrame(train_varthre,columns=X_train.columns[2:][varthre.get_support()])],1)
X_test=pd.concat([X_test.iloc[:,:2],pd.DataFrame(test_varthre,columns=X_test.columns[2:][varthre.get_support()])],1)

X_train.shape,X_test.shape,y_train.shape,submission.shape

In [ ]:
display(X_train.head(3))
print(X_train.shape)
display(X_test.head(3))
print(X_test.shape)

In [ ]:
genes=[col for col in X_train.columns if col.startswith("g-")]
cells=[col for col in X_train.columns if col.startswith("c-")]

In [ ]:
from sklearn.cluster import KMeans
# kmeans=KMeans(n_clusters=40,random_state=982)
kmeans=KMeans(n_clusters=35,random_state=982)
kmeans.fit(pd.concat([X_train[genes],X_test[genes]]))
X_train["cluster_g"]=kmeans.labels_[:len(X_train)]
X_test["cluster_g"]=kmeans.labels_[len(X_train):]

from sklearn.cluster import KMeans
# kmeans=KMeans(n_clusters=8,random_state=982)
kmeans=KMeans(n_clusters=5,random_state=982)
kmeans.fit(pd.concat([X_train[cells],X_test[cells]]))
X_train["cluster_c"]=kmeans.labels_[:len(X_train)]
X_test["cluster_c"]=kmeans.labels_[len(X_train):]

X_train=pd.get_dummies(X_train,columns=['cluster_g','cluster_c'])
X_test=pd.get_dummies(X_test,columns=['cluster_g','cluster_c'])

In [ ]:
gsquarecols=['g-574','g-211','g-216','g-0','g-255','g-577','g-153','g-389','g-60','g-370','g-248','g-167','g-203','g-177','g-301','g-332','g-517','g-6','g-744','g-224','g-162','g-3','g-736','g-486','g-283','g-22','g-359','g-361','g-440','g-335','g-106','g-307','g-745','g-146','g-416','g-298','g-666','g-91','g-17','g-549','g-145','g-157','g-768','g-568','g-396']

In [ ]:
for df in X_train,X_test:
    df['time_dose']=df['cp_time']+df['cp_dose']
    df['g_median']=df[genes].median(axis=1)
    df['c_median']=df[cells].median(axis=1)
    df['gc_median']=df[genes+cells].median(axis=1)
    df['g_max']=df[genes].max(axis=1)
    df['c_max']=df[cells].max(axis=1)
    df['gc_max']=df[genes+cells].max(axis=1)
    df['g_min']=df[genes].min(axis=1)
    df['c_min']=df[cells].min(axis=1)
    df['gc_min']=df[genes+cells].min(axis=1)
    df['g_sum']=df[genes].sum(axis=1)
    df['c_sum']=df[cells].sum(axis=1)
    df['gc_sum']=df[genes+cells].sum(axis=1)
    df['g_mean']=df[genes].mean(axis=1)
    df['c_mean']=df[cells].mean(axis=1)
    df['gc_mean']=df[genes+cells].mean(axis=1)
    df['g_std']=df[genes].std(axis=1)
    df['c_std']=df[cells].std(axis=1)
    df['gc_std']=df[genes+cells].std(axis=1)
    df['g_skew']=df[genes].skew(axis=1)
    df['c_skew']=df[cells].skew(axis=1)
    df['gc_skew']=df[genes+cells].skew(axis=1)
    df['g_kurtosis']=df[genes].kurtosis(axis=1)
    df['c_kurtosis']=df[cells].kurtosis(axis=1)
    df['gc_kurtosis']=df[genes+cells].kurtosis(axis=1)
    df['g_q25']=df[genes].quantile(0.25,axis=1)
    df['c_q25']=df[cells].quantile(0.25,axis=1)
    df['gc_q25']=df[genes+cells].quantile(0.25,axis=1)
    df['g_q75']=df[genes].quantile(0.75,axis=1)
    df['c_q75']=df[cells].quantile(0.75,axis=1)
    df['gc_q75']=df[genes+cells].quantile(0.75,axis=1)
    
    
    df['c52_c42'] = df['c-52'] * df['c-42']
    df['c13_c73'] = df['c-13'] * df['c-73']
    df['c26_c13'] = df['c-23'] * df['c-13']
    df['c33_c6'] = df['c-33'] * df['c-6']
    df['c11_c55'] = df['c-11'] * df['c-55']
    df['c38_c63'] = df['c-38'] * df['c-63']
    df['c38_c94'] = df['c-38'] * df['c-94']
    df['c13_c94'] = df['c-13'] * df['c-94']
    df['c4_c52'] = df['c-4'] * df['c-52']
    df['c4_c42'] = df['c-4'] * df['c-42']
    df['c13_c38'] = df['c-13'] * df['c-38']
    df['c55_c2'] = df['c-55'] * df['c-2']
    df['c55_c4'] = df['c-55'] * df['c-4']
    df['c4_c13'] = df['c-4'] * df['c-13']
    df['c82_c42'] = df['c-82'] * df['c-42']
    df['c66_c42'] = df['c-66'] * df['c-42']
    df['c6_c38'] = df['c-6'] * df['c-38']
    df['c2_c13'] = df['c-2'] * df['c-13']
    df['c62_c42'] = df['c-62'] * df['c-42']
    df['c90_c55'] = df['c-90'] * df['c-55']
    
    for feature in cells:
             df[f'{feature}_squared'] = df[feature] ** 2     
                
    for feature in gsquarecols:
        df[f'{feature}_squared'] = df[feature] ** 2

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

In [ ]:
X_train=X_train.loc[trt_cp_index,:]
y_train=y_train.loc[trt_cp_index,:]
y_train_non=y_train_non.loc[trt_cp_index,:]

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_train_non.shape)

In [ ]:
# def getmodel(layers,dropouts,activations1,activations2,layer_sizes1,layer_sizes2,lr,wd):
#     model=tf.keras.Sequential()
#     model.add(tf.keras.Input(shape=(689,)))
    
    
#     for i in range(layers):
#         model.add(
#             tf.keras.layers.BatchNormalization()
#         )
#         model.add(
#             tf.keras.layers.Dropout(dropouts[i])
#         )
#         model.add(
#             tfa.layers.WeightNormalization(tf.keras.layers.Dense(layer_sizes1[i],activation=activations1[i]))
#         )
#         model.add(
#             tf.keras.layers.Dense(layer_sizes2[i],activation=activations2[i])
#         )
        
        
#     model.add(tf.keras.layers.Dense(206,activation='softmax'))
    
# #     model.compile(optimizer=tfa.optimizers.AdamW(lr = 1e-3, weight_decay = 1e-5, clipvalue = 756),loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
#     model.compile(optimizer=tfa.optimizers.AdamW(lr = lr, weight_decay = wd, clipvalue = 756),loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
    
#     return model


# def get_model(layers,units,dropouts,activations):
#     model=tf.keras.Sequential()
#     model.add(tf.keras.Input(shape=(874,)))
#     model.add(tf.keras.layers.BatchNormalization())
        
#     for i in range(layers):
#         model.add(tf.keras.layers.Dense(units[i], activation=activations[i]))
#         model.add(tf.keras.layers.Dropout(dropouts[i]))
#         model.add(tf.keras.layers.BatchNormalization())
            
#     model.add(tf.keras.layers.Dense(206,activation='softmax'))
    
#     model.compile(optimizer = tf.keras.optimizers.Adam(1e-3),loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
#     return model

def create_model(num_cols, hid_layers, activations, dropout_rate, lr, num_cols_y):
    
    inp1 = tf.keras.layers.Input(shape = (num_cols, ))
    x1 = tf.keras.layers.BatchNormalization()(inp1)

    for i, units in enumerate(hid_layers):
#         x1 = tfa.layers.WeightNormalization(tf.keras.layers.Dense(units, activation=activations[i]))(x1)
        x1 = tf.keras.layers.Dense(units, activation=activations[i])(x1)
        x1 = tf.keras.layers.Dropout(dropout_rate[i])(x1)
        x1 = tf.keras.layers.BatchNormalization()(x1)
    
    x1 = tf.keras.layers.Dense(num_cols_y,activation='sigmoid')(x1)
    model = tf.keras.models.Model(inputs= inp1, outputs= x1)
    
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lr),loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.001), metrics=['accuracy'])
#     model.compile(optimizer=tfa.optimizers.AdamW(lr = 1e-3, weight_decay = 1e-5, clipvalue = 756),loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
    
    return model

In [ ]:
hps={
    30:[
    {
        "units":[1410,1246,512,512,512],
        "activations":['selu', 'swish', 'elu','elu','elu'],
        "dropouts":[0.49,0.51,0.0,0.0,0.0]
    },
    {
        "units":[1127,1133, 1209, 512],
        "activations":[ 'selu','swish','swish','elu'],
        "dropouts":[0.32,0.02,0.26,0.0]
    },
    {
        "units":[796,1995,573],
        "activations":['selu','selu','swish'],
        "dropouts":[0.22,0.21,0.09]
    },
    {
        "units":[959,1557,1826],
        "activations":['selu','selu','elu'],
        "dropouts":[0.42,0.61,0.3]
    },
    {
        "units":[1081,2033],
        "activations":['elu','elu'],
        "dropouts":[0.44,0.12]
    }    
    ],
    58:[
        {
            "units":[561,882,2023],
            "activations":['elu','elu','elu'],
            "dropouts":[0.43,0.17,0.29]
        },
        {
            "units":[687,1085,1365,1427,1325],
            "activations":['selu','swish','swish','elu','selu'],
            "dropouts":[0.59,0.15,0.31,0.08,0.41]
        },
        {
            "units":[1324,1475,976,512],
            "activations":['selu','selu','selu', 'elu'],
            "dropouts":[0.39,0.38,0.19,0.0]
        },
        {
            "units":[1410,1246,512,512,512],
            "activations":['selu', 'swish', 'elu', 'elu', 'elu'],
            "dropouts":[0.49,0.51,0.0,0.0,0.0]
        },
        {
            "units":[750,1734,638,512],
            "activations":['selu','swish','selu','elu'],
            "dropouts":[0.42,0.5,0.42,0.0]
        }
    ],
    3255:[
        {
            "units":[1444,629,512,512,512],
            "activations":['selu', 'swish', 'elu', 'elu', 'elu'],
            "dropouts":[0.62,0.4,0.0,0.0,0.0]
        },
        {
            "units":[899,940,893,718],
            "activations":['elu', 'elu', 'selu', 'selu'],
            "dropouts":[0.33,0.28,0.29,0.17]
        },
        {
            "units":[2030,1422],
            "activations":['elu', 'selu'],
            "dropouts":[0.53,0.37]
        },
        {
            "units":[1194,1562],
            "activations":[ 'swish','swish'],
            "dropouts":[0.59,0.0]
        },
        {
            "units":[1130,766,520,512],
            "activations":[  'selu','elu','elu', 'elu'],
            "dropouts":[0.29,0.14,0.27,0.0]
        }
    ]
}


In [ ]:
from pickle import load,dump

In [ ]:
y_train_non=y_train_non.iloc[:,y_train_non.columns!='sig_id']
final_y=pd.concat([y_train,y_train_non],1)
final_pred=pd.DataFrame(np.zeros((submission.shape[0], final_y.shape[1])),columns=final_y.columns)

for seed in [30,58,3255]:
    for n,(tr,te) in enumerate(MultilabelStratifiedKFold(n_splits=5, random_state=seed, shuffle=True).split(X_train,y_train)):
        print("SEED:"+str(seed)+" Fold:"+str(n))
        

        units=hps[seed][n]['units']
        activations=hps[seed][n]['activations']
        dropouts=hps[seed][n]['dropouts']
        
        model=create_model(X_train.shape[1], units, activations, dropouts, 1e-3, final_y.shape[1])
        

        model.fit(X_train.values[tr],
                 final_y.values[tr],
                 batch_size=128,
                 epochs=150,
                 validation_data=(X_train.values[te],final_y.values[te]),
                 verbose=1,
                  callbacks=[
                      tf.keras.callbacks.ReduceLROnPlateau(
                          monitor='val_loss', 
                          factor=0.1, 
                          patience=3,
                          epsilon = 1e-4, 
                          mode = 'min',
                          verbose=1
                      )
                      ,

                      tf.keras.callbacks.EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          mode='auto',
                          verbose=1,
                          baseline=None,
                          restore_best_weights=True
                      )
                  ]
                 )
        model.save(f'pSEED{seed}FOLD{n}.h5')
        model=create_model(X_train.shape[1], units, activations, dropouts, 1e-3, final_y.shape[1])
        model.load_weights(f'pSEED{seed}FOLD{n}.h5')
        final_pred+=model.predict(X_test)

In [ ]:
y_pred=final_pred.loc[:,submission.columns[1:]]/15

<h1>TRANSFER LEARNING START</h1>

In [ ]:
############################### HPS for transfer learning ##############################################
hps={
    30:[
    {
        "units":[512,1187, 2048, 1971],
        "activations":['elu', 'swish', 'selu','swish'],
        "dropouts":[0.28,0.26,0.47,0.6]
    },
    {
        "units":[1127,532, 1488, 1219],
        "activations":['swish', 'selu','elu','swish'],
        "dropouts":[0.57,0.0,0.16,0.44]
    },
    {
        "units":[1123,640, 512],
        "activations":['elu','swish','elu'],
        "dropouts":[0.14,0.64,0.0]
    },
    {
        "units":[1110,1248,588, 971],
        "activations":['selu','selu','elu','selu'],
        "dropouts":[0.29,0.61,0.64,0.41]
    },
    {
        "units":[1687,1494,1513],
        "activations":['elu','swish','elu'],
        "dropouts":[0.17,0.02,0.66]
    }    
    ],
    58:[
        {
            "units":[1395,1309],
            "activations":['selu','elu'],
            "dropouts":[0.51,0.36]
        },
        {
            "units":[897,1321],
            "activations":['elu', 'swish'],
            "dropouts":[0.59,0.63]
        },
        {
            "units":[1235,841],
            "activations":['selu', 'elu'],
            "dropouts":[0.69,0.05]
        },
        {
            "units":[1628,1272,512],
            "activations":['elu', 'swish', 'elu'],
            "dropouts":[0.63,0.44,0.0]
        },
        {
            "units":[646,1056,1827,774,1463],
            "activations":['selu','elu','swish','selu','selu'],
            "dropouts":[0.51,0.62,0.47,0.35,0.65]
        }
    ],
    3255:[
        {
            "units":[1148,1108,512,512],
            "activations":['selu', 'swish', 'elu', 'elu'],
            "dropouts":[0.28,0.69,0.0,0.0]
        },
        {
            "units":[1060,778,1430],
            "activations":['elu', 'selu', 'elu'],
            "dropouts":[0.68,0.44,0.66]
        },
        {
            "units":[1742,1453,1871,1655],
            "activations":['selu', 'swish', 'swish', 'selu'],
            "dropouts":[0.46,0.57,0.5,0.61]
        },
        {
            "units":[1788,1817,512,512,512,512],
            "activations":[ 'swish', 'elu', 'elu','elu','elu','elu'],
            "dropouts":[0.68,0.54,0.0,0.0,0.0,0.0]
        },
        {
            "units":[907,764],
            "activations":[  'elu', 'elu'],
            "dropouts":[0.47,0.38]
        }
    ]
}

In [ ]:
final_pred=submission.iloc[:,1:].copy()
final_pred.loc[:,:]=0

In [ ]:
for seed in [30,58,3255]:
    for n,(tr,te) in enumerate(MultilabelStratifiedKFold(n_splits=5, random_state=seed, shuffle=True).split(X_train,y_train)):
        print("SEED:"+str(seed)+" Fold:"+str(n))
        
        units=hps[seed][n]['units']
        activations=hps[seed][n]['activations']
        dropouts=hps[seed][n]['dropouts']
        
        
        model2=create_model(X_train.shape[1], units, activations, dropouts, 1e-3,y_train_non.shape[1])
        
        model2.fit(X_train.values[tr],
         y_train_non.values[tr],
         batch_size=128,
         epochs=150,
         validation_data=(X_train.values[te],y_train_non.values[te]),
         verbose=1,
          callbacks=[
              tf.keras.callbacks.ReduceLROnPlateau(
                  monitor='val_loss', 
                  factor=0.1, 
                  patience=3,
                  epsilon = 1e-4, 
                  mode = 'min',
                  verbose=1
              )
              ,
              
              tf.keras.callbacks.EarlyStopping(
                  monitor='val_loss',
                  min_delta=0,
                  patience=10,
                  mode='auto',
                  verbose=1,
                  baseline=None,
                  restore_best_weights=True
              )
          ]
         )

        model=create_model(X_train.shape[1], units, activations, dropouts, 1e-3, y_train.shape[1])
        for i in range(len(model.layers)-1):
            model.layers[i].set_weights(model2.layers[i].get_weights())

        model.fit(X_train.values[tr],
                 y_train.values[tr],
                 batch_size=128,
                 epochs=150,
                 validation_data=(X_train.values[te],y_train.values[te]),
                 verbose=1,
                  callbacks=[
                      tf.keras.callbacks.ReduceLROnPlateau(
                          monitor='val_loss', 
                          factor=0.1, 
                          patience=3,
                          epsilon = 1e-4, 
                          mode = 'min',
                          verbose=1
                      )
                      ,

                      tf.keras.callbacks.EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          mode='auto',
                          verbose=1,
                          baseline=None,
                          restore_best_weights=True
                      )
                  ]
                 )
        model.save(f'tSEED{seed}FOLD{n}.h5')
        model=create_model(X_train.shape[1], units, activations, dropouts, 1e-3, y_train.shape[1])
        model.load_weights(f'tSEED{seed}FOLD{n}.h5')
        
        final_pred+=model.predict(X_test)

In [ ]:
y_pred_transfer=final_pred/15
# y_pred=y_pred/2.35

In [ ]:
from sklearn.metrics import log_loss
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import roc_auc_score, log_loss

class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
        return np.mean(-aux)

In [ ]:
tabnet_params = dict(
    n_d = 32,
    n_a = 32,
    n_steps = 1,
    gamma = 1.3,
    lambda_sparse = 0,
    optimizer_fn = optim.Adam,
    optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5),
    mask_type = "entmax",
    scheduler_params = dict(
        mode = "min", patience = 5, min_lr = 1e-5, factor = 0.9),
    scheduler_fn = ReduceLROnPlateau,
    seed = 456,
    verbose = 10
)

In [ ]:
final_pred=submission.iloc[:,1:].copy()
final_pred.loc[:,:]=0

for seed in [30,58,3255]:
    for n,(tr,te) in enumerate(MultilabelStratifiedKFold(n_splits=5, random_state=seed, shuffle=True).split(X_train,y_train)):
        print("SEED:"+str(seed)+" Fold:"+str(n))
        
        model = TabNetRegressor(**tabnet_params)
        
        model.fit(
        X_train = X_train.values[tr],
        y_train = y_train.values[tr],
        eval_set = [(X_train.values[te],y_train.values[te])],
        eval_name = ["val"],
        eval_metric = ["logits_ll"],
        max_epochs = 200,
        patience = 20,
        batch_size = 512, 
        virtual_batch_size = 32,
        num_workers = 1,
        drop_last = False,
        # To use binary cross entropy because this is not a regression problem
        loss_fn = F.binary_cross_entropy_with_logits)
        
 
        model.save_model(f"tabSEED{seed}FOLD{n}")
        
        final_pred+=1 / (1 + np.exp(-model.predict(X_test.values)))

In [ ]:
y_pred_tabnet=final_pred/15

In [ ]:
y_pred_final=y_pred * 0.25 + y_pred_transfer * 0.25 + y_pred_tabnet * 0.5

In [ ]:
submission.head()

In [ ]:
submission.iloc[:,1:]=y_pred_final

In [ ]:
submission.loc[x_test_checkpoint["cp_type"]=='ctl_vehicle','5-alpha_reductase_inhibitor':]=0

In [ ]:
submission.head()

In [ ]:
submission.sum(1)

In [ ]:
submission.to_csv('submission.csv',index=False)